# Prediction of COVID-19 around the world

Student: Angela Amador

TMU Student Number: 500259095

Supervisor: Tamer Abdou, PhD


I aim to demonstrate how Machine Learning (ML) models were able to predict the spread of COVID-19 around the world.

First, I will explore the dataset to get insides and better understand patterns, detect error and outliers, and find relationships between variables. 


## Preparation
This dataset is taken from Our World in Data website, officially collected by Our World in Data team: https://covid.ourworldindata.org/data/owid-covid-data.csv.

This dataset will be synced daily. For more info: https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from ydata_profiling import ProfileReport
from IPython.core.interactiveshell import InteractiveShell
from numpy import unique

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"


### Load file and explore data

The dataset, provided by Our World in Data, provides COVID-19 information collected by Our World in Data available to Kaggle community https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset/download?datasetVersionNumber=418. This dataset is updated daily, for the purpose of this study I am analyzing the data with information up to Oct 7th, 2023.

In [ ]:
# Load file
raw_data = pd.read_csv('archive.zip', sep=',')  

#Explore data
raw_data.head()

### Check the data type and metadata of the attributes

In [ ]:
raw_data.dtypes

In [ ]:
# look at meta information about data, such as null values
raw_data.info()

In [ ]:
# Let's see meta information about numeric data, we can also see if there any extreme values
raw_data.describe()

### Removing data before COVID vaccine availability

Multiple vaccinates became available on the second semester of 2020. By December most countries have approved vaccinates for their own country. 

Vaccinations changed the behaviour of the pandemic then I will remove data before Jan 1st, 2021 to consider data only after vaccines became availability

In [ ]:
print("Original dataset:")
print("Total number of observations: ", raw_data.shape[0])
print("Total number of attributes: ", raw_data.shape[1])
print("Size: ", raw_data.size)


post_vaccine_data = raw_data.drop(raw_data[raw_data.date < '2021-01-01'].index)

print("\nAfter removing data before vaccinate was available around the world (Jan 1st, 2021):")
print("Total number of observations: ", post_vaccine_data.shape[0])
print("Total number of attributes: ", post_vaccine_data.shape[1])
print("Size: ", post_vaccine_data.size)


### Data Splitting
One of the first decisions to make is how to utilize the existing data. One common technique is to split the data into two groups typically referred to as the training and testing sets. The training set is used to develop
models and feature sets; it is the substrate for estimating parameters, comparing models, and all of the other activities required to reach a final model. The test set is used only at the conclusion of these activities for estimating a final, unbiased assessment of the model’s performance. It is critical that the test set not be used prior to this point. Looking at the test set results would bias the outcomes since the testing data will have become part of the model development process. Reference: Feature Engineering and Selection A Practical Approach for Predictive Models

In [ ]:
# Find out the index of total_cases column
c = post_vaccine_data.columns.get_loc('total_cases')

# Find out number of columns
d = post_vaccine_data.shape[1]

# Build feature, target arrays
X, y = post_vaccine_data.iloc[:, [i for i in range(d) if i != c]], post_vaccine_data.iloc[:, [c]]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1121218)

X_train
y_train
X_test
y_test

print("\nX_train dataset:")
print("Total number of observations: ", X_train.shape[0])
print("Total number of attributes: ", X_train.shape[1])
print("Size: ", X_train.size)

## Data Cleaning

### Identify Columns That Contain a Single Value
The dataset doesn't have columns with a single value

In [ ]:
# get number of unique values for each column
counts = X_train.nunique()

# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)

# drop useless columns
X_train.drop(to_del, axis=1, inplace=True)
print(X_train.shape)


### Consider Columns That Have Very Few Values
Even though we have 23 columns with less than 1% of unique values, most of them are numerical. In addition, some of the columns are categorical.

In [ ]:
# record columns to delete, columns with unique values less than 1 percent of rows
counts
to_del = [i for i,v in enumerate(counts) if (float(v)/X_train.shape[0]*100) < 1]
print(to_del)

# drop useless columns
#X_train.drop(to_del, axis=1, inplace=True)
#print(X_train.shape)

### Removing data columns with too many NaN values

We can calculate the ratio of missing values using a simple formula. The formula is the number of missing values in each column divided by the total number of observation. Generally, we can drop variables having a missing value ratio of more than 60% or 70%. For my purpose I am going to use a threashold of 60% missing values and remove those attributes.

In [ ]:
# Defining threashold of 60% missing values 
threashold_NaN = 0.60

#Explore data
def describe_nan(df):
    return pd.DataFrame([(i, df[df[i].isna()].shape[0],df[df[i].isna()].shape[0]/df.shape[0]) for i in df.columns], columns=['column', 'nan_counts', 'nan_rate'])

pd.options.display.max_rows = None

#icu=raw_data.icu_patients.value_counts(dropna=False)
#display ("NaN entries for the icu_patients column:", icu[icu.index.isnull()])

print("Attributes with more than 60 percentage of missing values:")

describe_nan(X_train).sort_values(by="nan_rate", ascending=False).query("nan_rate >= %s"%threashold_NaN)

#((raw_data.isnull() | raw_data.isna()).sum() * 100 / raw_data.index.size).round(2)

In [ ]:
drop_columns_NaN = describe_nan(X_train).sort_values(by="nan_rate", ascending=False).query("nan_rate >= %s"%threashold_NaN)[["column"]]
drop_columns_NaN = drop_columns_NaN['column'].to_list() 

# Removing data columns with too many missing values
# drop_columns_NaN
X_train_NaN = X_train.drop(drop_columns_NaN, axis=1, inplace=False)

print("After removing columns with more than 60 percentage of missing values:\n")
print("Total number of observations: ", X_train_NaN.shape[0])
print("Total number of attributes: ", X_train_NaN.shape[1])
print("Size: ", X_train_NaN.size)
print("\n")
X_train_NaN.info()

In [ ]:
print("Percentage of NaN values per attribute for the remaining columns:\n")
describe_nan(X_train_NaN).sort_values(by="nan_rate", ascending=False)

# del(dr1_data)

In [ ]:
X_train_NaN.head()
y_train.head()

## 2. Low Variance Filter

Another way of measuring how much information a data column has, is to measure its variance. In the limit case where the column cells assume a constant value, the variance would be 0 and the column would be of no help in the discrimination of different groups of data.

The Low Variance Filter node calculates each column variance and removes those columns with a variance value below a given threshold. Notice that the variance can only be calculated for numerical columns, i.e. this dimensionality reduction method applies only to numerical columns. Note, too, that the variance value depends on the column numerical range. Therefore data column ranges need to be normalized to make variance values independent from the column domain range.

First a Normalizer node normalizes all column ranges to [0, 1]; next, a Low Variance Filter node calculates the columns variance and filters out the columns with a variance lower than a set threshold.

In [ ]:
# Initialization is just like any other Scikit-learn estimator. The default value for the threshold is always 0. 
# Also, the estimator only works with numeric data obviously and it will raise an error if there are categorical features present in the dataframe. 
# That’s why, for now, I will subset the numeric features into another dataframe:

vt = VarianceThreshold()

#dr2 -> Dimensionality Reduction - 2. Removing low variance filter
dr2_data_num = dr1_data.select_dtypes(include="number")
#dr2_data_num.shape
#dr2_data_num.info()


In [ ]:
# Before, I need to take care of missing values encoded as NaN natively by replacing with the mean on reduced dataset "dr2_data_reduced"

print ("Before replacing NaN values with the mean:\n")
print("Total number of observations: ", dr2_data_num.shape[0])
print("Total number of attributes: ", dr2_data_num.shape[1])
print("Size: ", dr2_data_num.size)
print("\n")
dr2_data_num.info()

for c in dr2_data_num.columns:
    dr2_data_num[c] = dr2_data_num[c].fillna(dr2_data_num[c].mean())

print ("\nAfter replacing NaN values with the mean:\n")
print("Total number of observations: ", dr2_data_num.shape[0])
print("Total number of attributes: ", dr2_data_num.shape[1])
print("Size: ", dr2_data_num.size)
print("\n")
dr2_data_num.info()

In [ ]:
# First, we fit the estimator to data and call its get_support() method. It returns a boolean mask with True values for columns which are not dropped. 
# We can then use this mask to subset our DataFrame like so

_ = vt.fit(dr2_data_num)
mask = vt.get_support()

dr2_data_num = dr2_data_num.loc[:, mask]

# dr2_data_num.shape

# dr2_data_num.info()


In [ ]:
# We still have the same number of features. Now, let’s drop features with variances close to 0
vt = VarianceThreshold(threshold=1)

# Fit
_ = vt.fit(dr2_data_num)

# # Get the boolean mask
mask = vt.get_support()

dr2_data_reduced = dr2_data_num.loc[:, mask]

print ("\nAfter dropping features with variances close to 0:\n")
print("Total number of observations: ", dr2_data_reduced.shape[0])
print("Total number of attributes: ", dr2_data_reduced.shape[1])
print("Size: ", dr2_data_reduced.size)
print("\n")
dr2_data_reduced.info()

# With a threshold of 1, 3 attributes were removedthreshold
# From: (255173, 32)
# To: (255173, 29)

In [ ]:
# The attributes that were dropped are:
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

In [ ]:
# Method of normalizing all features by dividing them by their mean

normalized_df = dr2_data_num / dr2_data_num.mean()
normalized_df.head()

print("Variance of the normalized dataset:\n")
normalized_df.var()

In [ ]:
# Now, we can use the estimator with a lower threshold like 0.005
vt = VarianceThreshold(threshold=0.005)

# Fit
_ = vt.fit(normalized_df)

# # Get the boolean mask
mask = vt.get_support()

dr2_data_final = dr2_data_num.loc[:, mask]

dr2_data_final.shape

# With a threshold of 0.05, zero attributes were removed threshold
# From: (255173, 32)
# To: (255173, 32)

In [ ]:
# dr2_data_reduced.columns.get_loc('total_cases')
# dr2_data_reduced.shape[1]

In [ ]:
# With method of normalizing no attributes were removed; while with variances close to 0, 3 features were removed.
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

# I will check if it is rigth to removed these 3 attributes. I will test this by training two RandomForestRegressor to predict a total_cases: the first one on the reduced dataset (dr2_data_reduced), feature selected dataset
# and the second one on the full, numeric-feature only dataset (dr2_data_num).

#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import train_test_split

# Find out the index of total_cases column
c = dr2_data_reduced.columns.get_loc('total_cases')

# Find out number of columns
d = dr2_data_reduced.shape[1]

# Build feature, target arrays
X, y = dr2_data_reduced.iloc[:, [i for i in range(d) if i != c]], dr2_data_reduced.iloc[:, [c]]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1121218)

# Init, fit, score
forest = RandomForestRegressor(random_state=1121218)

_ = forest.fit(X_train, y_train)

# Training Score
print(f"Training Score: {forest.score(X_train, y_train)}")
#Training Score: 0.9999950801210624

print(f"Test Score: {forest.score(X_test, y_test)}")
# Test Score: 0.9999399219055796

print("Both training and test score suggest a really high performance without overfitting.")

In [ ]:
# dr2_data_num.columns.get_loc('total_cases')
# dr2_data_num.shape[1]

In [ ]:
# Now, let’s train the same model on the full numeric-only dataset

# Find out the index of total_cases column
c = dr2_data_num.columns.get_loc('total_cases')

# Find out number of columns
d = dr2_data_num.shape[1]


# Build feature, target arrays
X, y = dr2_data_num.iloc[:, [i for i in range(d) if i != c]], dr2_data_num.iloc[:, [c]]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1121218)

# Init, fit, score
forest = RandomForestRegressor(random_state=1121218)

_ = forest.fit(X_train, y_train)

# Training Score
print(f"Training Score: {forest.score(X_train, y_train)}")
#Training Score: 0.9999920034215055

print(f"Test Score: {forest.score(X_test, y_test)}")
# Test Score: 0.9999035126774154

print("I can confirm that there isn't any impact on the prediction by removing these 3 features")

#Freeing memory
del(X)
del(y)
del(X_train)
del(X_test)
del(y_train)
del(y_test)
# del(dr2_data_num)
# del(dr2_data_reduced)
# del(dr2_data_final)
del(normalized_df)

In [ ]:
# Droping the columns identified with variance close to 0
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

dr2_data = dr1_data.drop(['reproduction_rate', 'new_people_vaccinated_smoothed_per_hundred', 'human_development_index'], axis=1)
print("After removing columns identified with variance close to 0:\n")
print("Total number of observations: ", dr2_data.shape[0])
print("Total number of attributes: ", dr2_data.shape[1])
print("Size: ", dr2_data.size)
print("\n")
dr2_data.info()

## 3. High correlation with other data columns


* https://www.kaggle.com/code/bbloggsbott/feature-selection-correlation-and-p-value

### Selecting columns based on correlation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# Dataset copy to be use in the correlation, remove the column total_cases because it is the column we are trying to predict
#data = raw_data.drop(['total_cases'], axis=1)
dr3_data_corr = dr2_data.copy()

# The numpy.random.seed() makes the random numbers predictable and is used for reproducibility
np.random.seed(123)

# Find out the index for categorical variables
continent = dr3_data_corr.columns.get_loc('continent')
location = dr3_data_corr.columns.get_loc('location')
date = dr3_data_corr.columns.get_loc('date')

# Encode the Categorical Variable
# The dataset has 3 categorical attributes: date, continent and location
label_encoder = LabelEncoder()
dr3_data_corr.iloc[:,continent] = label_encoder.fit_transform(dr3_data_corr.iloc[:,continent]).astype('float64')
dr3_data_corr.iloc[:,location] = label_encoder.fit_transform(dr3_data_corr.iloc[:,location]).astype('float64')
dr3_data_corr.iloc[:,date] = label_encoder.fit_transform(dr3_data_corr.iloc[:,date]).astype('float64')

corr = dr3_data_corr.corr()
corr

sns.heatmap(corr)

In [ ]:
# Next, compare the correlation between features and remove one of two features that have a correlation higher than 0.9

columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

selected_columns = dr3_data_corr.columns[columns]
# selected_columns
# selected_columns.shape

## Add total_cases column which is the column to be predicted
total_cases = pd.Index(['total_cases'])
selected_columns = selected_columns.append(total_cases)
selected_columns
selected_columns.shape


In [ ]:
dr3_data_corr = dr3_data_corr[selected_columns]

In [ ]:
print ("After removing one of two features that have a correlation higher than 0.9:\n")
print("Total number of observations: ", dr3_data_corr.shape[0])
print("Total number of attributes: ", dr3_data_corr.shape[1])
print("Size: ", dr3_data_corr.size)
print("\n")
dr3_data_corr.info()

### Selecting columns based on p-value

Selecting the columns based on how they affect the p-value. 

Column total_cases because was removed, this is the column to be predicted

In [ ]:
# Removing the total_cases column
selected_columns = selected_columns[0:-1].values

In [ ]:
# Take care of missing values encoded as NaN natively by replacing with the mean 

for c in dr3_data_corr.columns:
    dr3_data_corr[c] = dr3_data_corr[c].fillna(dr3_data_corr[c].mean())


In [ ]:
import statsmodels.api as sm
def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    #x=np.array(x, dtype=float)
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    
    return x, columns, regressor_OLS.summary()

In [ ]:
# dr3_data_corr.head()
# dr3_data_corr.iloc[:,:-1].values 
# dr3_data_corr.iloc[:,-1].values
# selected_columns

In [ ]:
SL = 0.05
data_modeled, selected_columns, summary = backwardElimination(
    dr3_data_corr.iloc[:,:-1].values ,
    dr3_data_corr.iloc[:,-1].values,
    SL,
    selected_columns)


In [ ]:
# data_modeled
# selected_columns
summary

In [ ]:
result = pd.DataFrame()
result['total_cases'] = dr3_data_corr.iloc[:,-1]

dr3_data = pd.DataFrame(data = data_modeled, columns = selected_columns)

# dr3_data['total_cases'] = result['total_cases'] 

dr3_data.head()

In [ ]:

fig = plt.figure(figsize = (20, 25))
j = 0
for i in dr3_data.columns:
    plt.subplot(7, 4, j+1)
    j += 1
    sns.distplot(dr3_data[i])
    #plt.legend(loc='best',fontsize=10)
fig.suptitle('Covid Total Cases Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.95)


plt.show();

# Generate Profiling Report

In [ ]:
# Create the final dataset with the selected columns

selected_columns = np.append(selected_columns, 'total_cases')
selected_columns
selected_columns.shape

dr3_data_final = pd.DataFrame(data = dr2_data, columns = selected_columns)

print ("Final dataset:\n")
print("Total number of observations: ", dr3_data_final.shape[0])
print("Total number of attributes: ", dr3_data_final.shape[1])
print("Size: ", dr3_data_final.size)
print("\n")
dr3_data_final.info()


In [ ]:
# Genetate profiling report
#profile = ProfileReport(dr3_data_final, title="Profiling Report")
profile = ProfileReport(dr3_data_final, title="Profiling Report", html={'style':{'fullwith':True}})
profile